# Generación de Texto usando LSTM

Adaptado de https://www.kaggle.com/code/shivamb/beginners-guide-to-text-generation-using-lstms/notebook


**Autor:** Jazna Meza Hidalgo

**Correo Electrónico:** ja.meza@profesor.duoc.cl

**Fecha de Adaptación:** Junio 2025

**Versión:** 1.0  



---

## Descripción


Este notebook implementa una RNN para generación de texto.

---

## Requisitos de Software

Este notebook fue desarrollado con Python 3.9. A continuación se listan las bibliotecas necesarias:

-

Para verificar la versión instalada ejecutar usando el siguiente comando, usando la librería de la cual quieres saber la versión:

```bash
import pandas as pd
print(pd.__version__)
````



En este cuaderno, explicaremos cómo crear un modelo de lenguaje para generar texto en lenguaje natural mediante la implementación y el entrenamiento de una red neuronal recurrente de última generación.

### Generación de titulares de noticias

En este kernel, utilizaremos el conjunto de datos de [Comentarios y titulares del New York Times](https://www.kaggle.com/aashita/nyt-comments) para entrenar un modelo de lenguaje de generación de texto que se puede utilizar para generar titulares de noticias

## 1. Importar las bibliotecas

Como primer paso, debemos importar las bibliotecas necesarias:

In [2]:
import keras
import tensorflow as tf

# Configurar semillas para reproducibilidad
import numpy as np
import random
tf.random.set_seed(2)  # Corrige la llamada obsoleta
np.random.seed(1)
random.seed(1)

import pandas as pd
import string
import os

# Configurar warnings para evitar mensajes innecesarios
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)


## 2. Cargar los datos

Cargar los datos desde los titulares

In [19]:
!mkdir -p Articulos

In [20]:
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesApril2017.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesApril2018.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesFeb2017.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesFeb2018.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesJan2017.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesJan2018.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesMarch2017.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesMarch2018.csv
!wget -P Articulos https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesMay2017.csv

--2025-06-09 03:18:45--  https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesApril2017.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 429015 (419K) [text/plain]
Saving to: ‘Articulos/ArticlesApril2017.csv’

ArticlesApril2017.c 100%[===================>] 418.96K  --.-KB/s    in 0.07s   

2025-06-09 03:18:45 (6.14 MB/s) - ‘Articulos/ArticlesApril2017.csv’ saved [429015/429015]

--2025-06-09 03:18:45--  https://raw.githubusercontent.com/JaznaLaProfe/Deep-Learning/main/data/Articulos/ArticlesApril2018.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conne

In [21]:
curr_dir = '/content/Articulos'

all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(os.path.join(curr_dir, filename))
        all_headlines.extend(list(article_df.headline.values))

# Filtrar encabezados desconocidos
all_headlines = [h for h in all_headlines if h != "Unknown"]
print(f"Total de encabezados válidos: {len(all_headlines)}")


Total de encabezados válidos: 8603


## 3. Preparación del conjunto de datos

### 3.1 Limpieza del conjunto de datos

En el paso de preparación del conjunto de datos, primero realizaremos una limpieza del texto de los datos, que incluye la eliminación de signos de puntuación y el cambio a minúsculas de todas las palabras.

In [4]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal',
 'an antidote to europes populism',
 'the cost of a speech',
 'degradation of the language',
 'on the power of being awful',
 'trump garbles pitch on a revised health bill']

### 3.2 Generación de secuencias de tokens de N-gramas

El modelado del lenguaje requiere una secuencia de datos de entrada, ya que dada una secuencia (de palabras/tokens), el objetivo es predecir la siguiente palabra/token.

El siguiente paso es la tokenización. La tokenización es un proceso de extracción de tokens (términos/palabras) de un corpus. La biblioteca Keras de Python tiene un modelo incorporado para la tokenización que se puede utilizar para obtener los tokens y su índice en el corpus. Después de este paso, cada documento de texto en el conjunto de datos se convierte en una secuencia de tokens.


In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to sequence of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[46, 1601],
 [46, 1601, 1],
 [46, 1601, 1, 1951],
 [46, 1601, 1, 1951, 120],
 [122, 331],
 [122, 331, 1952],
 [122, 331, 1952, 2],
 [122, 331, 1952, 2, 125],
 [122, 331, 1952, 2, 125, 2484],
 [122, 331, 1952, 2, 125, 2484, 812]]

En la salida anterior, [30, 507], [30, 507, 11], [30, 507, 11, 1], etc., representan las frases ngram generadas a partir de los datos de entrada, donde cada entero corresponde al índice de una palabra particular en el vocabulario completo de palabras presentes en el texto. Por ejemplo

**Headline:** i stand  with the shedevils  
**Ngrams:** | **Sequence of Tokens**

<table>
<tr><td>Ngram </td><td> Sequence of Tokens</td></tr>
<tr> <td>i stand </td><td> [30, 507] </td></tr>
<tr> <td>i stand with </td><td> [30, 507, 11] </td></tr>
<tr> <td>i stand with the </td><td> [30, 507, 11, 1] </td></tr>
<tr> <td>i stand with the shedevils </td><td> [30, 507, 11, 1, 975] </td></tr>
</table>



### 3.3 Relleno de secuencias y obtención de variables: predictores y objetivo

Ahora que hemos generado un conjunto de datos que contiene una secuencia de tokens, es posible que las diferentes secuencias tengan longitudes diferentes. Antes de comenzar a entrenar el modelo, necesitamos rellenar las secuencias y hacer que sus longitudes sean iguales. Podemos usar la función pad_sequence de Keras para este propósito. Para ingresar estos datos en un modelo de aprendizaje, necesitamos crear predictores y etiquetas. Crearemos una secuencia de N-gramas como predictores y la siguiente palabra del N-grama como etiqueta. Por ejemplo:


Titular:  they are learning data science

<table>
<tr><td>PREDICTORS </td> <td>           LABEL </td></tr>
<tr><td>they                   </td> <td>  are</td></tr>
<tr><td>they are               </td> <td>  learning</td></tr>
<tr><td>they are learning      </td> <td>  data</td></tr>
<tr><td>they are learning data </td> <td>  science</td></tr>
</table>

In [7]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = keras.utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

Perfecto, ahora podemos obtener el vector de entrada X y el vector de etiqueta Y que se pueden utilizar para fines de entrenamiento.


## 4. LSTM para generación de texto

Diseñemos un modelo LSTM en nuestro código. Hemos agregado un total de tres capas al modelo.

1. Capa de entrada: toma la secuencia de palabras como entrada
2. Capa LSTM: calcula la salida utilizando unidades LSTM. Hemos añadido 100 unidades en la capa, pero este número se puede ajustar más adelante.
3. Capa de abandono: una capa de regularización que desactiva aleatoriamente las activaciones de algunas neuronas en la capa LSTM. Ayuda a evitar el sobreajuste. (Capa opcional)
4. Capa de salida: calcula la probabilidad de la mejor palabra siguiente posible como salida

Ejecutaremos este modelo durante un total de 100 épocas, pero se puede experimentar más

In [12]:
def create_model(max_sequence_len, total_words, model_name):
    input_len = max_sequence_len - 1
    model = tf.keras.models.Sequential(name=model_name)

    # Add Input Embedding Layer
    model.add(tf.keras.layers.Embedding(total_words, 10, input_length=input_len, name="Embedding"))

    # Add Hidden Layer 1 - LSTM Layer
    model.add(tf.keras.layers.LSTM(100, name="CapaLSTM"))
    model.add(tf.keras.layers.Dropout(0.1, name="Dropout"))

    # Add Output Layer
    model.add(tf.keras.layers.Dense(total_words, activation='softmax', name="Densa"))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

model = create_model(max_sequence_len, total_words, "generador")
model.summary()

Model: "generador"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ CapaLSTM (LSTM)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Densa (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss',
                           patience=3,  # Número de épocas sin mejora antes de detener
                           restore_best_weights=True)


In [13]:
history = model.fit(predictors, label, epochs=100)


Epoch 1/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 59s 35ms/step - loss: 8.0229
Epoch 2/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 58s 36ms/step - loss: 7.2757
Epoch 3/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 58s 36ms/step - loss: 7.0828
Epoch 4/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 83s 37ms/step - loss: 6.8969
Epoch 5/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 80s 36ms/step - loss: 6.7352
Epoch 6/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 81s 35ms/step - loss: 6.5788
Epoch 7/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 58s 36ms/step - loss: 6.4372
Epoch 8/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 85s 38ms/step - loss: 6.3459
Epoch 9/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 79s 36ms/step - loss: 6.2510
Epoch 10/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 80s 35ms/step - loss: 6.1559
Epoch 11/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 82s 35ms/step - loss: 6.0736
Epoch 12/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 86s 38ms/step - loss: 5.9310
Epoch 13/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 80s 37ms/step - loss: 5.7721
Epoch 14/100
1618/1618 ━━━━━━━━━━━━━━━━━━━━ 83s 38ms/step - 

In [15]:
model.save('modelo_generador.keras')

In [18]:
history_dict = history.history
loss_values = history_dict['loss']
loss_values

[7.85012674331665,
 7.339392185211182,
 7.077863693237305,
 6.862645626068115,
 6.686572074890137,
 6.533481597900391,
 6.391137599945068,
 6.366805076599121,
 6.2191267013549805,
 6.141563415527344,
 6.068075180053711,
 5.9187421798706055,
 5.822988033294678,
 5.785533905029297,
 5.586267471313477,
 5.483320236206055,
 5.306511878967285,
 5.1365532875061035,
 4.984583854675293,
 4.847666263580322,
 4.715892791748047,
 4.591485023498535,
 4.463147163391113,
 4.3534746170043945,
 4.23896598815918,
 4.1577019691467285,
 4.062850475311279,
 3.938730001449585,
 3.849604368209839,
 3.7620766162872314,
 3.6956889629364014,
 3.6212213039398193,
 3.5539278984069824,
 3.5062084197998047,
 3.4120304584503174,
 3.3520894050598145,
 3.29349946975708,
 3.260896682739258,
 3.1937718391418457,
 3.5954411029815674,
 3.907090187072754,
 3.5674450397491455,
 3.3938515186309814,
 3.2519474029541016,
 3.1424059867858887,
 3.180879592895508,
 3.0994369983673096,
 3.005850315093994,
 2.934981107711792,
 4.2

In [ ]:
model = tf.keras.models.load_model('modelo_generador.keras')

## 5. Generación del texto

Nuestra arquitectura de modelo ya está lista y podemos entrenarla usando nuestros datos. A continuación, escribamos la función para predecir la próxima palabra en función de las palabras de entrada (o texto inicial). Primero, tokenizaremos el texto inicial, rellenaremos las secuencias y lo pasaremos al modelo entrenado para obtener la palabra predicha. Las múltiples palabras predichas se pueden anexar para obtener la secuencia predicha.


In [16]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        # Convertir el texto semilla a una secuencia de tokens
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        # Predecir la probabilidad para cada palabra
        predicted_probs = model.predict(token_list, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)  # Obtener el índice de la palabra con la probabilidad más alta

        # Buscar la palabra correspondiente al índice predicho
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        # Añadir la palabra predicha al texto semilla
        seed_text += " " + output_word

    return seed_text.title()


## 6. Algunos resultados

In [17]:
print (generate_text("united states", 5, model, max_sequence_len))
print (generate_text("preident trump", 4, model, max_sequence_len))
print (generate_text("donald trump", 4, model, max_sequence_len))
print (generate_text("india and china", 4, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))

United States March Follow A Box And
Preident Trump Is A Way To
Donald Trump Is A Bear Lines
India And China Services May Be A
New York Today A Plethora Of
Science And Technology Limits On The Cameras Of


## Desafíos

Como podemos ver, el modelo ha producido un resultado que parece bastante bueno. Los resultados se pueden mejorar aún más con los siguientes puntos:
- Agregar más datos
- Ajustar la arquitectura de la red
- Ajustar los parámetros de la red